# Manual Shadow calculation from LIDAR

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import rasterio as rio

import zipfile
from osgeo import gdal
import requests
import io
from IPython.display import Image
from ipyleaflet import Map, ImageOverlay

from tqdm import tqdm
import pvlib
import pandas as pd

## Load Data

### DOM: Digitales Oberflächen Modell

This was the first attempt at getting some data and casting shadows. Unfortunately since the DOM is one connected mesh, the shadow quality isn't great but it renders fast. We could use this raster as a placeholder to test the entire workflow.

Link to the Berlin DOM data: https://fbinter.stadt-berlin.de/fb/berlin/service_intern.jsp?id=a_dom1@senstadt&type=FEED

In [44]:
# Download a file
def download_file(url, local_filename):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    return local_filename

tile = "386_5818"
link_to_tile = f"https://fbinter.stadt-berlin.de/fb/atom/DOM/DOM1_{tile}.zip"

def process_with_translate(input_file, dst_srs='EPSG:4326'):
        # 1. Convert to GeoTIFF
        intermediate_tif = "intermediate.tif"  # output from gdal_translate
        gdal.Translate(intermediate_tif, input_file, format='GTiff')

        # Run GDALwarp to convert from EPSG:25833 projection to ESPG:4326
        output_tif = "output.tif"  # output from gdalwarp
        gdal.Warp(output_tif, intermediate_tif, srcSRS='EPSG:25833', dstSRS=dst_srs)

        # 3. Apply DEM processing
        hillshade_tif = "hillshade.tif"  # output from DEMProcessing
        ds = gdal.DEMProcessing(hillshade_tif, output_tif, 'hillshade')
        gdal.DEMProcessing("")
        return ds

path_to_save = os.path.join(os.getcwd(), "data", f"tile{tile}.xyz")
f = download_file(link_to_tile, path_to_save)
process_with_translate(f, 'EPSG:3857')  # replace 'output.tif' with your preferred output filename

ERROR 4: `/home/unix_jz/github/shade-calculator/dev/shadowcalc/data/tile386_5818.xyz' not recognized as a supported file format.


ValueError: Received a NULL pointer.

ERROR 4: `/home/unix_jz/github/shade-calculator/dev/shadowcalc/data/tile386_5818.xyz' not recognized as a supported file format.


ValueError: Received a NULL pointer.

In [25]:
os.getcwd()

'/home/unix_jz/github/shade-calculator/dev/shadowcalc'

In [15]:
# Open the file
file_path = os.path.join('data', 'hillshade.tif')
with rio.open(file_path) as ds:
    img = ds.read(1)  # read the first (and only) band of the GeoTIFF

# Create a new figure
plt.figure(figsize=(10,10))
plt.xlim([0, 400])
plt.ylim([0, 400])
# Display the image
plt.imshow(img, cmap='gray')

# Optionally, add a colorbar
plt.colorbar(label='Digital Elevation Model')

# Show the plot
plt.show()

RasterioIOError: data/hillshade.tif: No such file or directory

### Airborne Laserscanning (ALS) Segmented Data


Direkter Download des Datensatzes (directly download the dataset)
Airborne Laserscanning (ALS) Primäre 3D Laserscan-Daten - Download-Service - EPSG:25833 - las, gezippt.
Erläuterung	Übersicht Downloadpakete, Flächenabdeckung je Datei 1 x 1 km
- Mitte (37,3 GB)	https://fbinter.stadt-berlin.de/lidar//Mitte.zip
- Nord (32,5 GB)	https://fbinter.stadt-berlin.de/lidar/Nord.zip
- Nordost (1 GB)	https://fbinter.stadt-berlin.de/lidar/Nordost.zip
- Nordwest (16,3 GB)	https://fbinter.stadt-berlin.de/lidar/Nordwest.zip
- Ost (16,1 GB)	https://fbinter.stadt-berlin.de/lidar/Ost.zip
- Süd (35,3 GB)	https://fbinter.stadt-berlin.de/lidar/Sued.zip
- Südost (50 GB)	https://fbinter.stadt-berlin.de/lidar/Suedost.zip
- Südwest (32 GB)	https://fbinter.stadt-berlin.de/lidar/Suedwest.zip
- West (27,3 GB)	https://fbinter.stadt-berlin.de/lidar/West.zip

In [6]:
# Run this line to view Berlin Tiles:
# Image(url='https://fbinter.stadt-berlin.de/fb/atom/DOP/Blattschnitt2x2km.gif')


In [20]:
class ALSdata():
    def __init__(self):
        self.set_paths()
        self.src_srs = "EPSG:25833"
        
    def set_paths(self):
        """Create a folder 
        in the same directory unless otherwise specified
        """
        self.data_path = os.path.join(os.getcwd(), "data")
        #file_path = os.path.join(data_path, f'temp_{filename}.xyz')

    def request_data_from_url(self, url):
        response = requests.get(url)
        assert response.status_code == 200
        self.response = response


    def download_file(self, url):
        self.local_filename = os.path.join(self.data_path, url.split("/")[-1])

        response = requests.get(url, stream=True)

        file_size = int(response.headers.get('Content-Length', 0))
        progress = tqdm(response.iter_content(1024), f'Downloading {self.local_filename}', total=file_size, unit='B', unit_scale=True, unit_divisor=1024)

        with open(self.local_filename, 'wb') as f:
            for data in progress.iterable:
                f.write(data)
                progress.update(len(data))


    def write_als_locally(self):
        with zipfile.ZipFile(self.local_filename, 'r') as zip_ref:
            for member in tqdm(zip_ref.infolist(), desc='Extracting '):
                zip_ref.extract(member, self.data_path)
        print(f"Extracted all files to directory: {os.path.abspath(self.data_path)}")


    def write_single_file_locally(self):
        with zipfile.ZipFile(response, 'r') as zip_ref:
            zip_ref.extractall(self.data_path)
        print(f"Extracted all files to directory: {os.path.abspath(self.data_path)}")


    def write_zip_locally(self, zip_file):
        unzip_path = os.path.join(self.data_path, zip_file[:-4])
        with zipfile.ZipFile(zip_file, "r") as z:
            for filename in z.namelist():
                # Open each file
                with z.open(filename) as f:
                    file_path = os.path.join(unzip_path, filename)
                    # Check if the file is a raster file by checking the extension
                    with open(file_path, 'wb') as temp_file:
                        temp_file.write(f.read())
    
    def process_with_translate(self, input_file, dst_srs='EPSG:4326'):
        # 1. Convert to GeoTIFF
        intermediate_tif = "intermediate.tif"  # output from gdal_translate
        gdal.Translate(intermediate_tif, input_file, format='GTiff')

        # Run GDALwarp to convert from EPSG:25833 projection to ESPG:4326
        output_tif = "output.tif"  # output from gdalwarp
        gdal.Warp(output_tif, intermediate_tif, srcSRS='EPSG:25833', dstSRS=dst_srs)

        # 3. Apply DEM processing
        hillshade_tif = "hillshade.tif"  # output from DEMProcessing
        ds = gdal.DEMProcessing(hillshade_tif, output_tif, 'hillshade')
        gdal.DEMProcessing("")
        return ds
        # Process the XYZ file with Translate
        #process_with_translate(file_path, 'EPSG:3857')  # replace 'output.tif' with your preferred output filename

    def process_superseded(self, r):
        #data_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "data")
        z = zipfile.ZipFile(io.BytesIO(r.content))
        for filename in z.namelist():
            file_path = os.path.join('data', f'temp_{filename}.xyz')
            print(filename)
            # Open each file
            with z.open(filename) as f:
                # Check if the file is a raster file by checking the extension
                with open(self.data_path, 'wb') as temp_file:
                    temp_file.write(f.read())
                # Process the XYZ file with Translate
                    #process_with_translate(file_path, 'EPSG:3857')  # replace 'output.tif' with your preferred output filename


        


In [12]:
nord_ost = ALSdata()
#nord_ost.download_file(nordost)
nord_ost.write_zip_locally("/home/unix_jz/github/shade-calculator/dev/shadowcalc/data/Nordost.zip")

nord_ost.write_als_locally()

NotImplementedError: That compression method is not supported

# Get Sun Location

In [83]:


latitude, longitude = 52.47, 13.4 # New York City
time = pd.date_range('2023-06-16 00:00:00', '2023-06-16 23:59:59', freq='H')  # One day with hourly frequency

# Create location object
location = pvlib.location.Location(latitude, longitude)

# Calculate solar position
solar_position = location.get_solarposition(time)
solar_position = solar_position.loc[solar_position["elevation"]>0,:]
# Print solar azimuth and zenith
print(solar_position[["elevation", "apparent_zenith"]])

                     elevation  apparent_zenith
2023-06-16 03:00:00   1.130931        88.520479
2023-06-16 04:00:00   8.793579        81.105593
2023-06-16 05:00:00  17.277196        72.669858
2023-06-16 06:00:00  26.249403        63.716746
2023-06-16 07:00:00  35.368432        54.607934
2023-06-16 08:00:00  44.214851        45.767870
2023-06-16 09:00:00  52.163359        37.823573
2023-06-16 10:00:00  58.187037        31.802524
2023-06-16 11:00:00  60.843918        29.146694
2023-06-16 12:00:00  59.174202        30.815756
2023-06-16 13:00:00  53.810501        36.177189
2023-06-16 14:00:00  46.206243        43.777634
2023-06-16 15:00:00  37.505666        52.472461
2023-06-16 16:00:00  28.410265        61.558825
2023-06-16 17:00:00  19.373718        70.579228
2023-06-16 18:00:00  10.748329        79.167666
2023-06-16 19:00:00   2.864723        86.902233
